# Quick Start Demo - AI Support Copilot

**5-minute demonstration** of the complete system.

Perfect for:
- Workshop introduction
- Quick demos
- Testing your setup

---

## Setup Required

**Terminal 1 - Start MCP Server:**
```bash
python -m support_mcp_server.server
```

**Environment:**
```bash
export OPENAI_API_KEY="your-key"
```

## 1. Imports & Setup

In [1]:
import json
from rich.console import Console
from rich.panel import Panel

from support_copilot_host.models import Ticket
from support_copilot_host.orchestrator import run_ticket_flow

console = Console()
console.print("[bold green]✓ Ready![/bold green]")

✓ Ready!

## 2. Load a Sample Ticket

In [2]:
# Load sample tickets
with open("data/tickets/samples.json") as f:
    samples = json.load(f)

# Use TICKET-001 (export timeout)
ticket = Ticket(**samples[0])

console.print(Panel(
    f"[cyan]ID:[/cyan] {ticket.id}\n\n"
    f"[cyan]Issue:[/cyan]\n{ticket.description}\n\n"
    f"[cyan]Logs:[/cyan]\n{ticket.log_snippet}",
    title="[bold]Incoming Support Ticket[/bold]",
    border_style="blue"
))

╭──────────────────────────────────────────── Incoming Support Ticket ────────────────────────────────────────────╮
│ ID: TICKET-001                                                                                                  │
│                                                                                                                 │
│ Issue:                                                                                                          │
│ Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout' after ~30       │
│ seconds.                                                                                                        │
│                                                                                                                 │
│ Logs:                                                                                                           │
│ 2025-11-01T09:21:10Z ERR_EXPORT_TIMEOUT workspace_id=ws_1234 service=export_service                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 3. Process Through AI System

This will:
1. **Triage** - Classify the issue
2. **Research** - Search docs, incidents, check status
3. **Action** - Draft customer reply + internal note
4. **Supervisor** - Review and approve

**⏱️ Takes ~15-20 seconds**

In [3]:
console.print("\n[bold yellow]🤖 Processing ticket through AI system...[/bold yellow]")
console.print("[dim](Running 4 agents with 4 LLM calls + 3 tool calls)[/dim]\n")

# Run the complete multi-agent flow
supervisor, trace = await run_ticket_flow(ticket)

console.print("[bold green]✓ Complete![/bold green]\n")

🤖 Processing ticket through AI system...

(Running 4 agents with 4 LLM calls + 3 tool calls)

✓ Complete!

## 4. View Results

In [4]:
# Customer-facing response
console.print(Panel(
    supervisor.final_customer_reply,
    title="[bold green]📧 Customer Reply[/bold green]",
    border_style="green",
    padding=(1, 2)
))

╭─────────────────────────────────────────────── 📧 Customer Reply ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Oh, I completely understand your frustration! The error message 'Export failed: timeout' you're seeing         │
│  indicates that the export process is taking longer than expected. This can occur when the workspace is         │
│  particularly large, causing delays in the export process.                                                      │
│                                                                                                                 │
│  In the meantime, you might want to check out our documentation on [Export to CSV                               │
│  Errors](docs/runbooks/export_to_csv_errors.md) for potential troubleshooting steps. If your workspace is       │
│  large, you could try exporting a smaller section or simplifying your dashboard.                                │
│                                                                                                                 │
│  We are aware of an ongoing incident (INC-1001) affecting CSV exports for larger workspaces, so you're not      │
│  alone in this. We're actively working on the issue.                                                            │
│                                                                                                                 │
│  Thank you for your patience as we resolve this issue!                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:
# Internal engineering note
console.print("\n")
console.print(Panel(
    supervisor.final_internal_note,
    title="[bold yellow]📝 Internal Note[/bold yellow]",
    border_style="yellow",
    padding=(1, 2)
))

╭─────────────────────────────────────────────── 📝 Internal Note ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Customer is experiencing a timeout error ('Export failed: timeout') when attempting to export their workspace  │
│  dashboard to CSV. This aligns with ongoing incident INC-1001, which addresses export timeouts for large        │
│  workspaces and is currently in a mitigating status.                                                            │
│                                                                                                                 │
│  We should suggest the customer refer to the documentation for 'Export to CSV Errors' for potential             │
│  troubleshooting steps. No immediate escalation required, but monitor the incident progress and follow up with  │
│  the customer as necessary.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
# Supervisor decision
console.print("\n")
if supervisor.approved:
    console.print("[bold green]✓ Supervisor Approved[/bold green]")
else:
    console.print("[bold yellow]⚠ Supervisor Made Edits[/bold yellow]")

console.print(f"[dim]Review notes: {supervisor.review_notes}[/dim]")

⚠ Supervisor Made Edits

Review notes: Removed informal language and metaphors from the customer reply to maintain professionalism and 
clarity. Simplified the message to focus on the issue and troubleshooting steps without unnecessary commentary.

## 5. Trace Summary

In [7]:
# Show what happened under the hood
agents_used = set(e.agent_name for e in trace.events if e.agent_name)
llm_calls = [e for e in trace.events if e.step_type == "LLM_CALL"]
tool_calls = [e for e in trace.events if e.step_type == "TOOL_CALL"]

console.print("\n[bold cyan]📊 What Happened:[/bold cyan]")
console.print(f"  • Agents involved: [yellow]{', '.join(sorted(agents_used))}[/yellow]")
console.print(f"  • LLM calls: [green]{len(llm_calls)}[/green]")
console.print(f"  • Tool calls: [blue]{len(tool_calls)}[/blue]")
console.print(f"  • Total events: [white]{len(trace.events)}[/white]")
console.print(f"  • Trace ID: [dim]{trace.trace_id}[/dim]")

📊 What Happened:

• Agents involved: ActionAgent, ResearchAgent, SupervisorAgent, TriageAgent

• LLM calls: 3

• Tool calls: 2

• Total events: 11

• Trace ID: ca1f1e4d-1146-4f49-ba60-66a34236e313

In [9]:
for event in trace.events:
    console.print(f"    - [{event.timestamp}] {event.agent_name} performed {event.step_type}")

- [2025-11-15T11:39:41.077752Z] TriageAgent performed LLM_CALL

- [2025-11-15T11:39:44.294025Z] TriageAgent performed AGENT_OUTPUT

- [2025-11-15T11:39:44.294082Z] None performed TOOL_CALL

- [2025-11-15T11:39:44.299587Z] None performed TOOL_RESULT

- [2025-11-15T11:39:44.299609Z] None performed TOOL_CALL

...

- [2025-11-15T11:39:44.302428Z] ResearchAgent performed AGENT_OUTPUT

- [2025-11-15T11:39:44.302475Z] ActionAgent performed LLM_CALL

- [2025-11-15T11:39:50.772454Z] ActionAgent performed AGENT_OUTPUT

- [2025-11-15T11:39:50.772517Z] SupervisorAgent performed LLM_CALL

- [2025-11-15T11:39:56.219847Z] SupervisorAgent performed AGENT_OUTPUT

## 6. Try Another Ticket Type

In [ ]:
# Try TICKET-002 (SSO authentication issue)
ticket2 = Ticket(**samples[1])

console.print("\n[bold magenta]Processing second ticket...[/bold magenta]")
console.print(Panel(
    f"{ticket2.description}\n\n[dim]{ticket2.log_snippet}[/dim]",
    title=f"[bold]{ticket2.id}[/bold]",
    border_style="blue"
))

supervisor2, trace2 = await run_ticket_flow(ticket2)

console.print("\n[bold green]✓ Complete![/bold green]\n")

In [ ]:
# Show customer reply
console.print(Panel(
    supervisor2.final_customer_reply,
    title="[bold green]Customer Reply[/bold green]",
    border_style="green"
))

## 7. Compare Tickets

In [ ]:
from rich.table import Table

# Compare the two tickets
comparison = Table(title="Ticket Comparison")
comparison.add_column("Metric", style="cyan")
comparison.add_column("TICKET-001", style="green")
comparison.add_column("TICKET-002", style="yellow")

comparison.add_row(
    "Issue Type",
    "Export timeout",
    "SSO authentication"
)
comparison.add_row(
    "Total Events",
    str(len(trace.events)),
    str(len(trace2.events))
)
comparison.add_row(
    "Tool Calls",
    str(len([e for e in trace.events if e.step_type == "TOOL_CALL"])),
    str(len([e for e in trace2.events if e.step_type == "TOOL_CALL"]))
)
comparison.add_row(
    "Approved",
    "✓" if supervisor.approved else "✗",
    "✓" if supervisor2.approved else "✗"
)

console.print(comparison)

console.print("\n[dim]💡 Notice: Different tickets trigger different research patterns![/dim]")